# Chapter 7. Your First Image Classifier

In [1]:
import cv2

In [2]:
class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        self.width = width
        self.height = height
        self.inter = inter
    def preprocess(self, image):
        return cv2.resize(image, (self.width, self.height),interpolation=self.inter) 
# cv2.INTER_AREA 用于收缩，cv2.INTER_CUBIC（慢）和 cv2.INTER_LINEAR 用于放大。

In [3]:
import numpy as np
import os
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        self.preprocessors = preprocessors
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, imagePaths, verbose=-1):
        data = []
        labels = []
        count=0
        for (i, imagePath) in enumerate(imagePaths):
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]   #/dataset_name/class/image.jpg
            if self.preprocessors is not None:
                for p in self.preprocessors:
                    if image is None:
                        os.remove(imagePath)
                        continue
                    image = p.preprocess(image)
                data.append(image)
                labels.append(label)
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print("[INFO] processed {}/{}".format(i + 1,
                    len(imagePaths)))
        return (np.array(data), np.array(labels))

In [4]:
dirdata='D:/mxnet/datasets/animals'
from imutils import paths
# imagePaths = list(paths.list_images(args["dataset"]))
imagePaths = list(paths.list_images(dirdata))

In [5]:
print(len(imagePaths))

3870


In [6]:
imagePaths[:10]

['D:/mxnet/datasets/animals\\cat\\0.jpg',
 'D:/mxnet/datasets/animals\\cat\\1.jpg',
 'D:/mxnet/datasets/animals\\cat\\10.jpg',
 'D:/mxnet/datasets/animals\\cat\\100.jpg',
 'D:/mxnet/datasets/animals\\cat\\1000.jpg',
 'D:/mxnet/datasets/animals\\cat\\1001.jpg',
 'D:/mxnet/datasets/animals\\cat\\1002.jpg',
 'D:/mxnet/datasets/animals\\cat\\1003.jpg',
 'D:/mxnet/datasets/animals\\cat\\1004.jpg',
 'D:/mxnet/datasets/animals\\cat\\1005.jpg']

In [7]:
imagePaths[1500:1510]

['D:/mxnet/datasets/animals\\dog\\10427.jpg',
 'D:/mxnet/datasets/animals\\dog\\10428.jpg',
 'D:/mxnet/datasets/animals\\dog\\10429.jpg',
 'D:/mxnet/datasets/animals\\dog\\1043.jpg',
 'D:/mxnet/datasets/animals\\dog\\10430.jpg',
 'D:/mxnet/datasets/animals\\dog\\10431.jpg',
 'D:/mxnet/datasets/animals\\dog\\10432.jpg',
 'D:/mxnet/datasets/animals\\dog\\10433.jpg',
 'D:/mxnet/datasets/animals\\dog\\10434.jpg',
 'D:/mxnet/datasets/animals\\dog\\10435.jpg']

In [8]:
imagePaths[3000:3010]

['D:/mxnet/datasets/animals\\panda\\n02510455_24.JPEG',
 'D:/mxnet/datasets/animals\\panda\\n02510455_240.JPEG',
 'D:/mxnet/datasets/animals\\panda\\n02510455_2440.JPEG',
 'D:/mxnet/datasets/animals\\panda\\n02510455_2449.JPEG',
 'D:/mxnet/datasets/animals\\panda\\n02510455_245.JPEG',
 'D:/mxnet/datasets/animals\\panda\\n02510455_2453.JPEG',
 'D:/mxnet/datasets/animals\\panda\\n02510455_2459.JPEG',
 'D:/mxnet/datasets/animals\\panda\\n02510455_2465.JPEG',
 'D:/mxnet/datasets/animals\\panda\\n02510455_2476.JPEG',
 'D:/mxnet/datasets/animals\\panda\\n02510455_2495.JPEG']

In [9]:
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp])

In [11]:
(data, labels) = sdl.load(imagePaths, verbose=500)

[INFO] processed 500/3870
[INFO] processed 1000/3870
[INFO] processed 1500/3870
[INFO] processed 2000/3870
[INFO] processed 2500/3870
[INFO] processed 3000/3870
[INFO] processed 3500/3870


In [12]:
print(len(data),len(labels))

3870 3870


In [15]:
print(data.shape,labels.shape)

(3870, 32, 32, 3) (3870,)


In [16]:
32*32*3

3072

In [17]:
data = data.reshape((data.shape[0], 3072))

In [18]:
data.shape

(3870, 3072)

In [19]:
data.nbytes

11888640

In [20]:
print("[INFO] features matrix: {:.4f}MB".format(
                        data.nbytes / (1024 * 1000.0)))

[INFO] features matrix: 11.6100MB


In [21]:
from sklearn.preprocessing import LabelEncoder
# LabelEncoder, a helper utility to convert labels represented as strings to integers
# where there is one unique integer per class label (a common practice when applying machine
# learning).
le = LabelEncoder()

In [22]:
labels[:5]

array(['cat', 'cat', 'cat', 'cat', 'cat'], dtype='<U5')

In [23]:
labels = le.fit_transform(labels)

In [24]:
labels[:5]

array([0, 0, 0, 0, 0], dtype=int64)

In [25]:
from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(data, labels,
            test_size = 0.25, random_state = 42)

In [26]:
print("[INFO] evaluating k-NN classifier...")

[INFO] evaluating k-NN classifier...


In [27]:
from sklearn.neighbors import KNeighborsClassifier
# model = KNeighborsClassifier(n_neighbors=args["neighbors"],
#     n_jobs = args["jobs"])
model = KNeighborsClassifier(n_neighbors=1,
    n_jobs = -1)

In [28]:
model.fit(trainX, trainY)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=1, p=2,
                     weights='uniform')

In [29]:
from sklearn.metrics import classification_report
print(classification_report(testY, model.predict(testX),
    target_names = le.classes_))

              precision    recall  f1-score   support

         cat       0.34      0.45      0.39       268
         dog       0.47      0.56      0.51       381
       panda       0.80      0.42      0.55       319

    accuracy                           0.48       968
   macro avg       0.54      0.48      0.48       968
weighted avg       0.55      0.48      0.49       968

